## Import statements for Dataframe manipulation

In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/RCV Voting Method Research/Preference Profiles/ResearchProfiles/WithoutWriteins')

ValueError: mount failed

# Create array of all files in working directory

In [ ]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]

Create test array

In [ ]:
files = ["PierceCounty08CountyExecMemberNoWriteins.csv"]

Create Dataframe for output

In [ ]:
output = pd.DataFrame(columns = ["Election", "Type", "Condorcet/Majority", "BCU", "QBC"])

Run eval on all files

In [ ]:
for f in files:
  data = pd.read_csv(f)
  ranks = data.drop(columns = ['Num. Voters'])
  candidates = ranks.stack().unique()
  for i in range(candidates.shape[0]):
    if candidates[i] == '-':
      candidates = np.delete(candidates, i)
      break
  data = data.to_numpy()
  data[0][0]

  row = [f]

  #condorcet/majority evalutation
  string = condorcetEval()
  info = string.split(":")
  row.append(info[0])
  row.append(info[1])

  #complete borda methods
  for m in ["borda", "quadraticComplete"]:
    points = np.full(len(candidates), 0.0)
    points = bordaCompleteEval(m)

    max = 0
    maxIndex = 0

    for i in range(points.shape[0]):
      if points[i] > max:
        max = points[i]
        maxIndex = i
    row.append(candidates[maxIndex])

  output.loc[len(output)] = row

output.to_csv("../Test.csv")

Define Complete Borda Evaluation Function

In [ ]:
def bordaCompleteEval(method):

  #function to control which counting method is being used and correct
  def countVotes(rank):
    if method == "divisibleComplete":
      return divisibleBorda(rank - 1)
    elif method == "exponentialComplete":
      return exponentialBorda(rank - 1)
    elif method == "borda":
      return borda(rank - 1)
    elif method  == "quadraticComplete":
      return quadraticBorda(rank - 1)

  #create array of points for each candidate
  points = np.full(len(candidates), 0.0)

  for i in range(data.shape[0]):
    if data[i][1] == '-':
      continue;
    votedCandidates = candidates.copy()
    lastColumn = 1
    for j in range(1, data.shape[1]):
      if data[i][j] == '-':

        #end loop to begin allocating remaining points
        break
      else:
        #allocate votes based on ranking
        for k in range(candidates.shape[0]):
          if data[i][j] == candidates[k]:
            votedCandidates[k] = 'voted'
            points[k] = points[k] + data[i][0]*countVotes(j)
        lastColumn = j + 1

    #calcualate remaining points
    remainder = 0
    for k in range(lastColumn, candidates.shape[0] + 1):
      remainder = remainder + countVotes(k)


    if method != "borda":
      #allocate remaining points equally among remaining candidates
      if candidates.shape[0] - lastColumn + 1 != 0:
        avg = remainder/(float(candidates.shape[0] - lastColumn + 1))
        for k in range(candidates.shape[0]):
          if candidates[k] == votedCandidates[k]:
            points[k] = points[k] + data[i][0]*avg

  return points

Define MBC Borda function

In [ ]:
def mbcBorda():

  for i in range(data.shape[0]):
    if data[i][1] == '-':
      continue;
    votedCandidates = []
    for j in range(1, data.shape[1]):
      if data[i][j] == '-':
        break
      else:
        votedCandidates.append(data[i][j])

    for c in range(len(votedCandidates)):
      for k in range(candidates.shape[0]):
        if votedCandidates[c] == candidates[k]:
          points[k] = points[k] + data[i][0]*(len(votedCandidates) - c)

  return points

Define Condorcet Evalution function

In [ ]:
def condorcetEval():

  ##create array of points for each candidate
  points = np.full(len(candidates), 0.0)

  ##allocate points based on first place positions
  for i in range(data.shape[0]):
    for index in range(candidates.shape[0]):
      if data[i][1] == candidates[index]:
        points[index] = points[index] + data[i][0]

  ##determine the candidate with most points
  sum = 0
  max = 0
  for i in range(points.shape[0]):
    sum = sum + points[i]
    if points[i] > max:
      max = points[i]
      maxIndex = i

  ##determine if there is a majority candidate
  if (points[maxIndex] + 0.0)/sum > 0.5:
    return "majority:" + candidates[maxIndex]

  lost = np.full(len(candidates), False)

  ##if not
  for i in range(candidates.shape[0]):
    for j in range(i + 1, candidates.shape[0]):
      a = 0
      b = 0
      for k in range(data.shape[0]):
        for l in range(data.shape[1]):
          if data[k][l] == candidates[i]:
            a = a + data[k][0]
            break
          if data[k][l] == candidates[j]:
            b = b + data[k][0]
            break
      if a > b:
        lost[j] = True
      else:
        lost[i] = True

  for i in range(candidates.shape[0]):
    if not lost[i]:
      return "condorcet:" + candidates[i]

  return "No Candidate:No Candidate"

Define borda count methods

In [ ]:
#Allocate points by index in array via borda count
def borda(rank):
  return candidates.shape[0] - rank

#Allocate points via divisible borda count
def divisibleBorda(rank):
  return 2*borda(rank) - 1

#Allocate points via fibonacci borda count
def fibonacci(n):
  return round(1/math.sqrt(5)*pow((1+math.sqrt(5))/2, n))

def fibonacciBorda(rank):
  return fibonacci(borda(rank) + 1)

#Allocate points via exponential borda count
def exponentialBorda(rank):
  return pow(2, borda(rank) - 1)

def quadraticBorda(rank):
  return borda(rank)*(borda(rank) - 1)/2 + 1

bordaEnum = ["borda", "divisible", "exponential", "fibonacci", "quandratic", "bordaComplete", "divisibleComplete", "exponentialComplete", "fibonacciComplete", "quadraticComplete"]
bordaEnum = enumerate(bordaEnum)